In [3]:
import pandas as pd

# Load the CSV files
umsatz_df = pd.read_csv('umsatzdaten_gekuerzt.csv')
kiwo_df = pd.read_csv('kiwo.csv')
wetter_df = pd.read_csv('wetter.csv')

# Display the first few rows and column names to understand the structure
umsatz_df.head(), kiwo_df.head(), wetter_df.head()

(        id       Datum  Warengruppe      Umsatz
 0  1307011  2013-07-01            1  148.828353
 1  1307021  2013-07-02            1  159.793757
 2  1307031  2013-07-03            1  111.885594
 3  1307041  2013-07-04            1  168.864941
 4  1307051  2013-07-05            1  171.280754,
         Datum  KielerWoche
 0  2012-06-16            1
 1  2012-06-17            1
 2  2012-06-18            1
 3  2012-06-19            1
 4  2012-06-20            1,
         Datum  Bewoelkung  Temperatur  Windgeschwindigkeit  Wettercode
 0  2012-01-01         8.0      9.8250                   14        58.0
 1  2012-01-02         7.0      7.4375                   12         NaN
 2  2012-01-03         8.0      5.5375                   18        63.0
 3  2012-01-04         4.0      5.6875                   19        80.0
 4  2012-01-05         6.0      5.3000                   23        80.0)